In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
df = pd.read_csv('raw_logs.csv',header=0).groupby('session')
for session, sdf in df:
    timeStr = datetime.fromtimestamp(session/1000.0).strftime('%Y-%m-%d %H:%M:%S')
    print('session {0} ({1}), {2} records'.format(session, timeStr, len(sdf.index)))
    
    # Sort by timestamp get the final average MPG for every trip (kff1205)
    sdf = sdf.sort_values('time')
    
    # Remove invalid values
    sdf = sdf[sdf.k43 != 0]
    
    # Key values
    ## k11     =  Throttle Position(Manifold)
    ## kc      =  Engine RPM
    ## kff1271 =  Fuel used (trip)
    ## kff1204 =  Trip Distance
    ## kff1205 =  Trip average MPG

session 1509673999804 (2017-11-02 20:53:19), 1678 records
session 1509715520133 (2017-11-03 08:25:20), 493 records
session 1509814879698 (2017-11-04 12:01:19), 468 records
session 1510237809601 (2017-11-09 08:30:09), 2361 records


In [15]:
sdf.sort_values('time').keys()

Index(['v', 'session', 'id', 'time', 'kff1005', 'kff1006', 'kff1001',
       'kff1007', 'k4', 'k2f', 'k11', 'k5', 'kc', 'kd', 'kf', 'kff1226',
       'kff1220', 'kff1221', 'k46', 'eml', 'k43', 'kff1271', 'k1f', 'kff1205',
       'kff1204', 'kff1266', 'kff1270'],
      dtype='object')